# Use the orchestration service of Generative AI Hub

The orchestration service of Generative AI Hub lets you use all the available models with the same codebase. You only deploy the orchestration service and then you can access all available models simply by changing the model name parameter. You can also use grounding, prompt templating, data masking and content filtering capabilities.

Store the `orchestration deployment url` from the previous step in your `variables.py` file. This code is based on the [AI180 TechEd 2024 Jump-Start session](https://github.com/SAP-samples/teched2024-AI180/tree/e648921c46337b57f61ecc9a93251d4b838d7ad0/exercises/python).

👉 Make sure you assign the deployment url of the orchestration service to `AICORE_ORCHESTRATION_DEPLOYMENT_URL` in [variables.py](variables.py).

In [1]:
import json
import os
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
# Inline credentials
with open('creds.json') as f:
    credCF = json.load(f)
 
# Set environment variables
def set_environment_vars(credCF):
    env_vars = {
        'AICORE_AUTH_URL': credCF['url'] + '/oauth/token',
        'AICORE_CLIENT_ID': credCF['clientid'],
        'AICORE_CLIENT_SECRET': credCF['clientsecret'],
        'AICORE_BASE_URL': credCF["serviceurls"]["AI_API_URL"] + "/v2",
        'AICORE_RESOURCE_GROUP': "llm-deployed"
    }
 
    for key, value in env_vars.items():
        os.environ[key] = value
        print(value)
 
# Create AI Core client instance
def create_ai_core_client(credCF):
    set_environment_vars(credCF)  # Ensure environment variables are set
    return AICoreV2Client(
        base_url=os.environ['AICORE_BASE_URL'],
        auth_url=os.environ['AICORE_AUTH_URL'],
        client_id=os.environ['AICORE_CLIENT_ID'],
        client_secret=os.environ['AICORE_CLIENT_SECRET'],
        resource_group=os.environ['AICORE_RESOURCE_GROUP']
    )
 
ai_core_client = create_ai_core_client(credCF)

https://anuragv2-39wjy902.authentication.eu10.hana.ondemand.com/oauth/token
sb-adcd4907-f1a7-462d-ba8e-646390ee4185!b398425|aicore!b540
107d3e9b-9a41-4b30-9b24-a091a45956cd$VmYxjzammFm50xkj1O37HmzgX3maoNrwfrlm99qUhi0=
https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2
llm-deployed


In [2]:
# import init_env
# import variables

# init_env.set_environment_variables()

### Import the packages you want to use

In [3]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.config import GroundingModule, OrchestrationConfig
from gen_ai_hub.orchestration.models.document_grounding import DocumentGrounding, DocumentGroundingFilter
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.service import OrchestrationService

### Assign a model and define a prompt template
**user_query** is again the user input. Whereas **grounding_response** is the context retrieved from the context information, in this case from sap.help.com.

In [4]:
AICORE_ORCHESTRATION_DEPLOYMENT_URL = "https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d56c73baa4a99356"

In [5]:
# TODO again you need to chose a model e.g. gemini-1.5-flash or gpt-4o-mini
llm = LLM(
    name="gpt-4o",
    parameters={
        'temperature': 0.0,
    }
)
template = Template(
            messages=[
                SystemMessage("You are a helpful translation assistant."),
                UserMessage("""Answer the request by providing relevant answers that fit to the request.
                Request: {{ ?user_query }}
                Context:{{ ?grounding_response }}
                """),
            ]
        )

### Create an orchestration configuration that specifies the grounding capability

In [6]:
# Set up Document Grounding
filters = [
            DocumentGroundingFilter(id="SAPHelp", data_repository_type="help.sap.com")
        ]

grounding_config = GroundingModule(
            type="document_grounding_service",
            config=DocumentGrounding(input_params=["user_query"], output_param="grounding_response", filters=filters)
        )

config = OrchestrationConfig(
    template=template,
    llm=llm,
    grounding=grounding_config
)

In [ ]:
orchestration_service = OrchestrationService(
    api_url=AICORE_ORCHESTRATION_DEPLOYMENT_URL,
    config=config
)

response = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="user_query",
            #TODO Here you can change the user prompt into whatever you want to ask the model
            value="What is Joule?"
        )
    ]
)

print(response.orchestration_result.choices[0].message.content)

# Streaming
Long response times can be frustrating in chat applications, especially when a large amount of text is involved. To create a smoother, more engaging user experience, we can use streaming to display the response in real-time, character by character or chunk by chunk, as the model generates it. This avoids the awkward wait for a complete response and provides immediate feedback to the user.

In [ ]:
response = orchestration_service.stream(
    config=config,
    template_values=[
        TemplateValue(
            name="user_query",
            #TODO Here you can change the user prompt into whatever you want to ask the model
            value="What is Joule?"
        )
    ]
)

for chunk in response:
    print(chunk.orchestration_result.choices[0].delta.content)


[More Info on the content filter](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/content-filtering)

[Next exercise](10-chatbot-with-memory.ipynb)